# Retrieval code 처음부터 따라가보기 

In [1]:
!nvidia-smi

Wed May  5 03:21:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   37C    P0    26W / 250W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Import libraries

In [2]:
import os, sys
from pathlib import Path
from glob import glob

In [3]:
BASE_PATH = Path('.').resolve().parent
BASE_PATH

PosixPath('/opt/ml/jaepil')

In [4]:
sys.path.append(BASE_PATH.as_posix())

In [5]:
sys.path

['/opt/ml/jaepil/code',
 '/opt/conda/envs/dl38/lib/python38.zip',
 '/opt/conda/envs/dl38/lib/python3.8',
 '/opt/conda/envs/dl38/lib/python3.8/lib-dynload',
 '',
 '/opt/conda/envs/dl38/lib/python3.8/site-packages',
 '/opt/conda/envs/dl38/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/opt/conda/envs/dl38/lib/python3.8/site-packages/IPython/extensions',
 '/opt/ml/.ipython',
 '/opt/ml/jaepil']

In [6]:
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm.auto import tqdm
import pandas as pd
import pickle
import json
import os
import numpy as np

from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)
from konlpy.tag import Mecab

import time
from contextlib import contextmanager

In [7]:
data_path = BASE_PATH / "input" / "data" / "data"
data_path

PosixPath('/opt/ml/jaepil/input/data/data')

In [8]:
train_path = data_path / "train_dataset"

train_data_path = train_path / "train"
val_data_path = train_path / "validation"

In [9]:
test_path = data_path / "test_dataset"

test_data_path = test_path / "validation"

## `__main__`

In [10]:
org_dataset = load_from_disk(train_path)
org_dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [11]:
full_ds = concatenate_datasets(
    [
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ]
) # train dev 를 합친 4192 개 질문에 대해 모두 테스트

Loading cached processed dataset at /opt/ml/jaepil/input/data/data/train_dataset/train/cache-fbc57aa6e699fb0c.arrow
Loading cached processed dataset at /opt/ml/jaepil/input/data/data/train_dataset/validation/cache-d2fba0c42123b1d6.arrow


In [12]:
full_ds # train + validation 모두에 대해 테스트 

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 4192
})

In [13]:
### Mecab 이 가장 높은 성능을 보였기에 mecab 으로 선택 했습니다 ###
mecab = Mecab()
def tokenize(text):
    # return text.split(" ")
    return mecab.morphs(text)

(from `__main__`)

```python
wiki_path = "wikipedia_documents.json"
retriever = SparseRetrieval(
    # tokenize_fn=tokenizer.tokenize,
    tokenize_fn=tokenize,
    data_path="data",
    context_path=wiki_path)
```

## `SparseRetrieval` class

### `.__init__()`

In [14]:
wiki_path = data_path / "wikipedia_documents.json"

tokenize_fn = tokenize
data_path = data_path
context_path = wiki_path

In [15]:
with open(context_path, 'r') as f:
    wiki = json.load(f)

In [16]:
contexts = list(dict.fromkeys([v['text'] for v in wiki.values()]))
len(contexts)

56737

In [17]:
contexts[0]

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'

In [18]:
ids = list(range(len(contexts)))

In [19]:
# Transform by vectorizer
tfidfv = TfidfVectorizer(
    tokenizer=tokenize_fn,
    ngram_range=(1, 2),
    max_features=50000
)

In [20]:
tfidfv

TfidfVectorizer(max_features=50000, ngram_range=(1, 2),
                tokenizer=<function tokenize at 0x7f67ce21d040>)

In [21]:
# should run get_sparse_embedding() or build_faiss() first.
p_embedding = None
indexer = None

(from `__main__`)

```python
# test single query
query = "대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?"

with timer("single query by exhaustive search"):
    scores, indices = retriever.retrieve(query)
with timer("single query by faiss"):
    scores, indices = retriever.retrieve_faiss(query)
```

### `.retrieve()`

In [22]:
query = "대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?" # It is string

In [23]:
isinstance(query, str)

True

In [24]:
topk = 1

```python
doc_scores, doc_indices = get_relevant_doc(query, k=topk)
```

#### `.get_relevant_doc()`

In [25]:
query_vec = tfidfv.transform([query])
query_vec

NotFittedError: The TF-IDF vectorizer is not fitted

### `.retrieve_faiss()`